In [7]:
import pandas as pd

cleaning/transformation for 1 table

In [ ]:
df = pd.read_csv("/Users/aman/Desktop/VS Code/DE Assignment (IC)/Data_generation/data/mock_csvs_final/user_activity_1.csv")


In [11]:
df.head(10)

,log_id,user_id,watch_time(min),timestamp,ip_address
0,3397568,781513,38,2025-06-15 11:31:39.045860,241
1,3091488,999290,158,2025-06-15 05:40:00.045860,54
2,3339791,415324,149,2025-06-15 17:30:52.045860,229
3,2411340,528072,42,2025-06-15 15:31:17.045860,36
4,8957928,2977,43,2025-06-15 13:17:01.045860,236
5,478840,977243,124,2025-06-15 17:28:48.045860,221
6,296336,741430,161,2025-06-15 11:43:20.045860,180
7,1493580,761877,65,2025-06-15 03:32:42.045860,33
8,2369838,899873,55,2025-06-15 17:54:09.045860,53
9,600507,425934,147,2025-06-14 23:16:23.045860,194


In [12]:
print(len(df))

10000000


In [13]:
unique_ids   = df["log_id"].nunique()
print(unique_ids)

6321604


In [14]:
unique_ids   = df["user_id"].nunique()
print(unique_ids)

999953


In [9]:
df_dedup = df.drop_duplicates(subset=["log_id"], keep="first")

In [10]:
len(df_dedup)

632269

In [12]:
df_dedup.head(10)

,log_id,user_id,watch_time(min),timestamp,ip_address
0,337619,57502,12,2025-06-15 05:17:45.812618,18
1,32944,5889,115,2025-06-14 20:13:23.812618,91
2,734258,95005,118,2025-06-15 10:05:51.812618,212
3,649312,31556,171,2025-06-15 02:11:18.812618,144
4,836893,19695,154,2025-06-14 14:45:59.812618,144
5,913725,18061,155,2025-06-15 06:15:50.812618,14
6,34336,72289,55,2025-06-14 23:26:58.812618,81
7,382655,11140,20,2025-06-14 18:08:49.812618,112
8,43629,35418,167,2025-06-15 04:09:55.812618,112
9,49284,48082,97,2025-06-15 10:07:39.812618,88


In [11]:
def map_ip_to_region(ip):
    # ip is an integer 1–256
    return f"Region_{int(ip)}"


In [14]:
df_dedup["geo_region"] = df_dedup["ip_address"].apply(map_ip_to_region)

/var/folders/8t/sqd_61ws1wng_py49tc67mpw0000gn/T/ipykernel_12435/2618229237.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dedup["geo_region"] = df_dedup["ip_address"].apply(map_ip_to_region)


In [15]:
df_dedup.head(10)

,log_id,user_id,watch_time(min),timestamp,ip_address,geo_region
0,337619,57502,12,2025-06-15 05:17:45.812618,18,Region_18
1,32944,5889,115,2025-06-14 20:13:23.812618,91,Region_91
2,734258,95005,118,2025-06-15 10:05:51.812618,212,Region_212
3,649312,31556,171,2025-06-15 02:11:18.812618,144,Region_144
4,836893,19695,154,2025-06-14 14:45:59.812618,144,Region_144
5,913725,18061,155,2025-06-15 06:15:50.812618,14,Region_14
6,34336,72289,55,2025-06-14 23:26:58.812618,81,Region_81
7,382655,11140,20,2025-06-14 18:08:49.812618,112,Region_112
8,43629,35418,167,2025-06-15 04:09:55.812618,112,Region_112
9,49284,48082,97,2025-06-15 10:07:39.812618,88,Region_88


In [ ]:
import pandas as pd

def dedupe_and_enrich(file_path: str) -> pd.DataFrame:
    """
    1) Reads the CSV at `file_path`
    2) Drops duplicate log_id rows (keeps the first occurrence)
    3) Adds a `geo_region` column (mapping ip_address → Region_x)
    4) Returns the transformed DataFrame
    """
    df = pd.read_csv(file_path)
    
    df_transformed = df.drop_duplicates(subset=["log_id"], keep="first").copy()
    
    # 3. Enriching geo_region (here assuming ip_address is an int 1–256)
    df_transformed["geo_region"] = df_transformed["ip_address"].apply(lambda ip: f"Region_{int(ip)}")
    
    user_totals = (
    df_transformed
    .groupby("user_id", as_index=False)["watch_time(min)"]
    .sum()
    .rename(columns={"watch_time(min)": "total_watch_time"})
)
    

    return user_totals


In [29]:
dedupe_and_enrich("/Users/aman/Desktop/VS Code/DE Assignment (IC)/Data_generation/data/mock_csvs/user_activity_1.csv")

,user_id,total_watch_time
0,1,331
1,2,323
2,3,338
3,4,590
4,5,400
...,...,...
99832,99995,577
99833,99996,454
99834,99997,885
99835,99998,218


In [30]:
dedupe_and_enrich("/Users/aman/Desktop/VS Code/DE Assignment (IC)/Data_generation/data/mock_csvs/user_activity_2.csv")

,user_id,total_watch_time
0,1,940
1,2,889
2,3,243
3,4,333
4,5,316
...,...,...
99821,99995,1244
99822,99996,522
99823,99997,431
99824,99998,338


In [ ]:
final_aggregate = pd.DataFrame()

for i in range(1,4):
    df = dedupe_and_enrich(f"/Users/aman/Desktop/VS Code/DE Assignment (IC)/Data_generation/data/mock_csvs/user_activity_{i}.csv")
    final_aggregate = pd.concat([final_aggregate, df], ignore_index=True)



In [34]:
print(final_aggregate)

        user_id  total_watch_time
0             1               331
1             2               323
2             3               338
3             4               590
4             5               400
...         ...               ...
299498    99995               401
299499    99996               775
299500    99997               842
299501    99998               966
299502    99999               420

[299503 rows x 2 columns]


In [36]:
user_totals = (
    final_aggregate
    .groupby("user_id", as_index=False)["total_watch_time"]
    .sum()
    .rename(columns={"total_watch_time": "total_watch_time_all"})
)

In [37]:
user_totals.head()

,user_id,total_watch_time_all
0,1,2003
1,2,1714
2,3,998
3,4,1529
4,5,1537


In [23]:
transformed_df = dedupe_and_enrich("/Users/aman/Desktop/VS Code/DE Assignment (IC)/Data_generation/data/mock_csvs/user_activity_1.csv")
# display(transformed_df.head(10))

In [26]:
user_totals = (
    transformed_df
    .groupby("user_id", as_index=False)["watch_time(min)"]
    .sum()
    .rename(columns={"watch_time(min)": "total_watch_time"})
)


In [27]:
user_totals.head(20)

,user_id,total_watch_time
0,1,331
1,2,323
2,3,338
3,4,590
4,5,400
5,6,981
6,7,641
7,8,491
8,9,413
9,10,746
